# 🎓 Spinoza Secours - Notebook Colab

**Modèle :** Mistral 7B + LoRA Fine-tuned  
**Prompt :** Système hybride optimisé (~250-300 tokens)  
**API :** FastAPI + ngrok


## 📦 Cellule 1 : Installation Dépendances


In [1]:
!pip install -q pyngrok fastapi uvicorn transformers peft accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.8 MB/s eta 0:00:00


## 📚 Cellule 2 : Imports + Configuration

**🔐 Configuration des secrets (à faire une seule fois) :**

1. Clique sur l'icône 🔑 **Secrets** dans le panneau de gauche de Colab
2. Ajoute deux secrets :
   - **`ngrok`** : Ton token ngrok (https://dashboard.ngrok.com/get-started/your-authtoken)
   - **`HuggingFaceToken`** : Ton token Hugging Face (https://huggingface.co/settings/tokens)

Les tokens seront récupérés automatiquement via `userdata.get()`.


In [2]:
import os
import re
import random
import threading
import torch
from typing import Dict, List, Optional
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import userdata

# Configuration modèle
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_MODEL = "FJDaz/mistral-7b-philosophes-lora"

# Récupération des tokens depuis Colab Secrets
# ⚠️ Configure tes secrets dans Colab : 🔑 Secrets → Ajouter 'ngrok' et 'HuggingFaceToken'
try:
    NGROK_TOKEN = userdata.get('ngrok')
    print("✅ Token ngrok récupéré depuis Colab Secrets")
except:
    print("❌ Token ngrok non trouvé ! Configure le secret 'ngrok' dans Colab Secrets")
    NGROK_TOKEN = None

try:
    HF_TOKEN = userdata.get('HuggingFaceToken')
    print("✅ Token Hugging Face récupéré depuis Colab Secrets")
except:
    print("❌ Token Hugging Face non trouvé ! Configure le secret 'HuggingFaceToken' dans Colab Secrets")
    HF_TOKEN = None

# Configuration ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("⚠️ ngrok ne sera pas configuré sans token")

# Gestion des ports : Port aléatoire pour éviter les conflits
PORT = random.randint(8001, 9000)
print(f"🔌 Port sélectionné : {PORT}")

# Tuer processus sur le port sélectionné si existant
import subprocess
try:
    result = subprocess.run(f"lsof -ti:{PORT}", shell=True, capture_output=True, text=True)
    if result.stdout.strip():
        subprocess.run(f"kill -9 {result.stdout.strip()}", shell=True)
        print(f"🧹 Port {PORT} libéré")
except:
    pass

print("✅ Imports et configuration OK")


✅ Token ngrok récupéré depuis Colab Secrets
✅ Token Hugging Face récupéré depuis Colab Secrets
🔌 Port sélectionné : 8755
✅ Imports et configuration OK


## 🎯 Cellule 3 : Prompt Système Hybride


In [3]:
# Prompt système hybride optimisé (~250 tokens)
SYSTEM_PROMPT_SPINOZA = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

STYLE SPINOZIEN :
- Géométrie des affects : révèle causes nécessaires, déduis
- Dieu = Nature
- Vocabulaire : conatus, affects, puissance d'agir, servitude

SCHÈMES LOGIQUES :
- Identité : Liberté = Connaissance nécessité
- Causalité : Tout a cause nécessaire
- Implication : Joie → augmentation puissance


MÉTHODE :
1. Révèle nécessité causale
2. Distingue servitude (ignorance) vs liberté (connaissance)
3. Exemples concrets modernes

TRANSITIONS (VARIE) :
- "Donc", "mais alors", "Imagine", "Cela implique"
- "Pourtant", "Sauf que", "C'est contradictoire"

RÈGLES :
- Tutoie (tu/ton/ta)
- Concis (2-3 phrases MAX)
- Questionne au lieu d'affirmer
- Tu ES Spinoza, pas un commentateur de Spinoza.
- Langage DÉTENDU : tu peux être un peu familier, c'est OK
- Évite les phrases trop longues ou compliquées

STYLE :
- Langage JEUNE : parle comme un prof cool, pas comme un livre
- Utilise des mots simples, des expressions d'aujourd'hui
- Tu peux dire "ouais", "ben", "genre", "en fait" - c'est naturel
- Réponds DIRECTEMENT à ce que l'élève vient de dire
- Utilise tes idées (causalité, nécessité, liberté = connaissance) mais avec des mots simples"""

INSTRUCTIONS_CONTEXTUELLES = {
    "confusion": "L'élève est confus → Donne UNE analogie concrète simple. Réponds DIRECTEMENT à sa confusion.",
    "resistance": "L'élève résiste → Révèle contradiction avec 'mais alors'. Réponds DIRECTEMENT à son objection.",
    "accord": "L'élève est d'accord → Valide puis AVANCE logiquement avec 'Donc'. Réponds à ce qu'il vient de dire.",
    "neutre": "Élève neutre → Pose question pour faire réfléchir. Réponds à ce qu'il dit."
}

# RAG DÉSACTIVÉ par défaut - trop embrouillant
INSTRUCTION_RAG = """
⚠️ RAG DÉSACTIVÉ - Réponds avec tes propres idées, pas en récitant.
"""

def construire_prompt_complet(contexte: str, use_rag_instruction: bool = True) -> str:
    """
    Construit le prompt complet optimisé

    Args:
        contexte: "accord", "confusion", "resistance", "neutre"
        use_rag_instruction: Si True, ajoute instructions RAG

    Returns:
        Prompt système complet (~250-300 tokens)
    """
    prompt = SYSTEM_PROMPT_SPINOZA

    # Ajouter instruction contextuelle
    if contexte in INSTRUCTIONS_CONTEXTUELLES:
        prompt += f"\n\n{INSTRUCTIONS_CONTEXTUELLES[contexte]}"

    # Ajouter instruction RAG (optionnel)
    if use_rag_instruction:
        prompt += f"\n\n{INSTRUCTION_RAG}"

    return prompt

print("✅ Prompt système hybride chargé")


✅ Prompt système hybride chargé


## 🔍 Cellule 4 : Détection Contexte + Post-Processing


In [4]:
def detecter_contexte(user_input: str) -> str:
    """Détecte le contexte de la réponse utilisateur"""
    text_lower = user_input.lower()

    # Accord
    if any(word in text_lower for word in ['oui', "d'accord", 'exact', 'ok', 'voilà', 'tout à fait']):
        return "accord"

    # Confusion
    if any(phrase in text_lower for phrase in ['comprends pas', 'vois pas', "c'est quoi", 'je sais pas', 'pourquoi', 'rapport']):
        return "confusion"

    # Résistance
    if any(word in text_lower for word in ['mais', 'non', "pas d'accord", 'faux', "n'importe quoi", 'je peux']):
        return "resistance"

    return "neutre"

def nettoyer_reponse(text: str) -> str:
    """Nettoie la réponse générée"""
    text = re.sub(r'\([^)]*[Aa]ttends[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Pp]oursuis[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Dd]onne[^)]*\)', '', text)
    text = re.sub(r'[😀-🙏🌀-🗿🚀-🛿]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+([.!?])', r'\1', text)
    return text

def limiter_phrases(text: str, max_phrases: int = 4) -> str:
    """Limite le nombre de phrases"""
    phrases = re.split(r'[.!?]+\s+', text)
    phrases = [p.strip() for p in phrases if p.strip()]
    if len(phrases) <= max_phrases:
        return text
    return '. '.join(phrases[:max_phrases]) + '.'

print("✅ Détection contexte + Post-processing OK")


✅ Détection contexte + Post-processing OK


## 🤖 Cellule 5 : Chargement Modèle

**⚠️ Code reproduit exactement depuis app.py - Ne pas modifier**


In [5]:
@torch.no_grad()
def load_model():
    has_gpu = torch.cuda.is_available()
    print(f"🖥️ GPU disponible: {has_gpu}")

    if has_gpu:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        device_map = "auto"
        torch_dtype = torch.bfloat16
    else:
        quantization_config = None
        device_map = "cpu"
        torch_dtype = torch.float32

    print(f"🔄 Chargement Mistral 7B ({'4-bit GPU' if has_gpu else 'FP32 CPU'})...")

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=quantization_config,
        device_map=device_map,
        torch_dtype=torch_dtype,
        token=HF_TOKEN,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    print("🔄 Chargement tokenizer...")

    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL,
        token=HF_TOKEN,
        trust_remote_code=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("🔄 Application LoRA Spinoza_Secours...")

    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_MODEL,
        token=HF_TOKEN
    )

    print("✅ Modèle Mistral 7B + LoRA chargé!")
    return model, tokenizer

# Charger le modèle
model, tokenizer = load_model()


🖥️ GPU disponible: True
🔄 Chargement Mistral 7B (4-bit GPU)...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

🔄 Application LoRA Spinoza_Secours...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

✅ Modèle Mistral 7B + LoRA chargé!


## 💬 Cellule 6 : Fonction spinoza_repond()

**Utilise le prompt système hybride optimisé**


In [6]:
# Historique conversation (format: [user, assistant])
conversation_history = []

def spinoza_repond(message: str) -> str:
    """
    Génère une réponse de Spinoza avec prompt hybride adaptatif

    Args:
        message: Message de l'utilisateur

    Returns:
        Réponse de Spinoza nettoyée
    """
    global conversation_history

    # Détecter contexte
    contexte = detecter_contexte(message)

    # Construire prompt adaptatif
    system_prompt = construire_prompt_complet(contexte, use_rag_instruction=False)

    # Formatage Mistral style
    prompt_parts = [f"<s>[INST] {system_prompt}\n\n"]

    # Ajouter historique (4 derniers échanges max)
    for entry in conversation_history[-4:]:
        prompt_parts.append(f"{entry[0]} [/INST] {entry[1]}</s>[INST] ")

    prompt_parts.append(f"{message} [/INST]")
    text = "".join(prompt_parts)

    # Tokenization
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    # Génération
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Décodage
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Post-processing
    response = nettoyer_reponse(response)
    response = limiter_phrases(response, max_phrases=3)

    # Mettre à jour historique
    conversation_history.append([message, response])

    return response

print("✅ Fonction spinoza_repond() créée")


✅ Fonction spinoza_repond() créée


## 🌐 Cellule 7 : API FastAPI + ngrok


In [7]:
# Questions d'amorce
QUESTIONS_BAC = [
    "La liberté est-elle une illusion ?",
    "Suis-je esclave de mes désirs ?",
    "Puis-je maîtriser mes émotions ?",
    "La joie procure-t-elle un pouvoir ?",
    "Peut-on désirer sans souffrir ?",
]

# Modèles Pydantic
class ChatRequest(BaseModel):
    message: str
    history: Optional[List[List[str]]] = None

class ChatResponse(BaseModel):
    reply: str
    history: List[List[str]]

# API FastAPI
app = FastAPI()

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # ⚠️ À restreindre en production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def root():
    """Endpoint racine - Informations sur l'API"""
    return {
        "name": "Spinoza Secours API",
        "model": "Mistral 7B + LoRA",
        "status": "running",
        "endpoints": {
            "health": "GET /health",
            "init": "GET /init",
            "chat": "POST /chat"
        }
    }

@app.get("/health")
def health():
    return {"status": "ok", "model": "Mistral 7B + LoRA"}

@app.get("/init")
def init():
    global conversation_history
    conversation_history = []
    question = random.choice(QUESTIONS_BAC)
    greeting = f"Bonjour ! Je suis Spinoza. Discutons :\n\n**{question}**\n\nQu'en penses-tu ?"
    return {
        "greeting": greeting,
        "history": [[None, greeting]]
    }

@app.post("/chat")
def chat(req: ChatRequest):
    global conversation_history

    # Mettre à jour historique si fourni
    if req.history:
        conversation_history = req.history

    # Générer réponse
    reply = spinoza_repond(req.message)

    return {
        "reply": reply,
        "history": conversation_history
    }

print("✅ API FastAPI créée")


✅ API FastAPI créée


## 🚀 Cellule 8 : Lancement Serveur + ngrok


In [8]:
# Lancer FastAPI en background avec port dynamique
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Attendre que le serveur démarre
import time
time.sleep(3)

# Tunnel ngrok sur le port dynamique
tunnel = ngrok.connect(PORT)
# Extraire l'URL publique depuis l'objet NgrokTunnel
# L'objet NgrokTunnel a une méthode public_url ou on peut extraire depuis str()
if hasattr(tunnel, 'public_url'):
    public_url = tunnel.public_url
elif hasattr(tunnel, 'data') and 'public_url' in tunnel.data:
    public_url = tunnel.data['public_url']
else:
    # Fallback : extraire depuis la représentation string
    import re
    url_match = re.search(r'https://[^"]+\.ngrok[^"]+', str(tunnel))
    public_url = url_match.group(0) if url_match else str(tunnel)

print("\n" + "=" * 80)
print(f"🚀 API PUBLIQUE : {public_url}")
print(f"📡 Health : {public_url}/health")
print(f"💬 Init : {public_url}/init")
print(f"💬 Chat : POST {public_url}/chat")
print("=" * 80)
print(f"\n✅ Serveur lancé sur le port {PORT} !")
print(f"📋 Copie cette URL dans index_spinoza.html : {public_url}")



🚀 API PUBLIQUE : https://nonremunerative-rory-unbreakably.ngrok-free.dev
📡 Health : https://nonremunerative-rory-unbreakably.ngrok-free.dev/health
💬 Init : https://nonremunerative-rory-unbreakably.ngrok-free.dev/init
💬 Chat : POST https://nonremunerative-rory-unbreakably.ngrok-free.dev/chat

✅ Serveur lancé sur le port 8755 !
📋 Copie cette URL dans index_spinoza.html : https://nonremunerative-rory-unbreakably.ngrok-free.dev


In [ ]:
# =============================================================================
# ⚖️ PROMPT D'ÉVALUATION (Calibré pour Mistral 7B)
# =============================================================================

PROMPT_EVALUATION = """Tu évalues un dialogue entre Spinoza et un élève.

DIALOGUE :
{dialogue}

GRILLE D'ÉVALUATION (0-10 pour chaque critère) :

1️⃣ COMPRÉHENSION :
   • 9-10 : Reformule correctement les idées, pose des questions pertinentes, fait des liens
   • 6-8 : Comprend mais demande des clarifications, fait quelques erreurs
   • 3-5 : Comprend partiellement, confusions fréquentes
   • 0-2 : Ne comprend pas, dit "je comprends pas", refuse le dialogue

2️⃣ COOPÉRATION :
   • 9-10 : Répond aux questions, fait avancer le dialogue, s'engage activement
   • 6-8 : Coopère mais avec quelques résistances (ex: "En voilà un pâté !")
   • 3-5 : Résiste souvent, dialogue difficile
   • 0-2 : Refuse de coopérer, dit "j'ai autre chose à faire", part

3️⃣ PROGRESSION :
   • 9-10 : Pensée évolue clairement, conclusions à la fin (ex: "Je comprends maintenant!")
   • 6-8 : Progresse lentement, quelques retours en arrière
   • 3-5 : Peu de progression, tourne en rond
   • 0-2 : Aucune évolution, répète les mêmes incompréhensions

⚠️ PRINCIPE BIENVEILLANT : Un élève qui questionne intelligemment, conteste avec arguments, ou relève des incohérences mérite une BONNE note (7-9).

EXEMPLES CONCRETS :
• "J'en ai rien à faire. Ciao." → comp=1, coop=1, prog=0
• "Je comprends pas" (répété) → comp=3, coop=5, prog=2
• "Ah oui ! c'est plus précis !" (progression visible) → comp=7, coop=8, prog=8
• "Je comprends maintenant. Merci !" → comp=9, coop=10, prog=10

Réponds UNIQUEMENT avec ce JSON (aucun texte avant/après) :

{{
 "comprehension": X,
 "cooperation": Y,
 "progression": Z,
 "total": X+Y+Z
}}"""

In [11]:
# =============================================================================
# ⚡ ÉVALUATION INCRÉMENTALE - Fil de l'Eau (Optimisation Inférence)
# =============================================================================
# 🎮 Cellule Colab : Évaluation Incrémentale Maïeuthon
# À ajouter APRÈS la cellule /evaluate (Maïeuthon) et AVANT la cellule Lancement Serveur
# =============================================================================

# Import nécessaire pour query parameter
from fastapi import Query

# Prompt évaluation incrémentale (court, rapide)
PROMPT_EVALUATION_INCREMENTAL = """Évalue rapidement ces 2 derniers échanges (0-10) :

{dialogue_recent}

CRITÈRES :
• Compréhension : L'élève comprend-il les idées ? (0=rien compris, 10=parfait)
• Coopération : Coopère-t-il activement ? (0=refuse, 10=très engagé)
• Progression : Sa pensée progresse-t-elle ? (0=aucune, 10=grande évolution)

EXEMPLES :
- "J'en ai rien à faire" → 1, 1, 0
- "Je comprends pas" → 3, 5, 2
- "Ah oui ! c'est plus précis !" → 7, 8, 7
- "Je comprends maintenant !" → 9, 10, 9

Réponds UNIQUEMENT avec ce JSON :

{{
 "comprehension": X,
 "cooperation": Y,
 "progression": Z,
 "total": X+Y+Z
}}"""

# Stockage scores incrémentaux (en mémoire)
# Structure : {dialogue_id: [scores_échange_2, scores_échange_4, ...]}
incremental_scores = {}

def evaluer_incremental(dialogue: str, debug: bool = False, return_raw: bool = False):
    """
    Évaluation légère au fil de l'eau (tous les 2 échanges)
    - Prompt court (2 derniers échanges seulement)
    - Température basse (0.1) - Strict pour JSON
    - Max tokens réduit (100) - Garantit un JSON complet
    - Pas de message final - Gain de temps

    Args:
        dialogue: Le dialogue à évaluer
        debug: Si True, affiche la réponse brute du modèle pour diagnostic (dans les logs Colab)
        return_raw: Si True, retourne aussi la réponse brute du modèle dans le tuple

    Returns:
        dict: Les scores évalués si return_raw=False
        tuple: (dict scores, str raw_response) si return_raw=True
    """
    # Extraire les 2 derniers échanges seulement (4 lignes : Élève + Spinoza x2)
    lines = [l.strip() for l in dialogue.split('\n') if l.strip()]
    if len(lines) > 4:
        recent_exchanges = '\n'.join(lines[-4:])  # 2 derniers échanges
    else:
        recent_exchanges = dialogue

    prompt_eval = PROMPT_EVALUATION_INCREMENTAL.format(dialogue_recent=recent_exchanges)

    # Formatage Mistral
    prompt_eval_formatted = f"<s>[INST] {prompt_eval} [/INST]"

    inputs = tokenizer(prompt_eval_formatted, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    # Inférence rapide (température basse, tokens réduits)
    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,  # Augmenté pour garantir un JSON complet (était 50)
            temperature=0.1,    # Strict pour JSON
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][input_length:]
    reponse_eval = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Debug: afficher la réponse brute si demandé
    if debug:
        print(f"🔍 [DEBUG] Réponse brute du modèle: {reponse_eval[:500]}")

    # Parser JSON (amélioré pour capturer même avec texte autour)
    # Essayer plusieurs stratégies de parsing
    details_model = None

    # Stratégie 1: Pattern spécifique avec tous les champs requis
    # Accepter les variantes avec/sans accents
    json_pattern = r'\{[^{}]*"(?:comprehension|compréhension|comprésentation)"[^{}]*"(?:cooperation|coopération)"[^{}]*"progression"[^{}]*"total"[^{}]*\}'
    json_match = re.search(json_pattern, reponse_eval, re.DOTALL)

    if json_match:
        try:
            json_str = json_match.group(0)
            # Nettoyer le JSON (enlever les retours à la ligne en trop, etc.)
            json_str = json_str.strip()
            details_model = json.loads(json_str)
        except json.JSONDecodeError as e:
            # Debug: afficher la réponse brute pour diagnostic
            if debug:
                print(f"⚠️ Erreur parsing JSON incrémental (stratégie 1): {e}")
                print(f"   JSON brut extrait: {json_match.group(0)[:300]}")

    # Stratégie 2: Si échec, chercher le premier bloc JSON valide
    if not details_model:
        json_pattern_fallback = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        json_match = re.search(json_pattern_fallback, reponse_eval, re.DOTALL)
        if json_match:
            try:
                json_str = json_match.group(0).strip()
                details_model = json.loads(json_str)
            except json.JSONDecodeError as e:
                if debug:
                    print(f"⚠️ Erreur parsing JSON incrémental (stratégie 2): {e}")

    # Stratégie 3: Si toujours échec, essayer de parser directement toute la réponse
    if not details_model:
        try:
            details_model = json.loads(reponse_eval.strip())
        except json.JSONDecodeError:
            if debug:
                print(f"⚠️ Erreur parsing JSON incrémental (stratégie 3): Impossible de parser la réponse")
                print(f"   Réponse complète: {reponse_eval[:300]}")

    # Valider et compléter le modèle
    if details_model and isinstance(details_model, dict):
        # Normaliser les clés (gérer les accents : "compréhension" -> "comprehension")
        # Mapping des variantes possibles vers les clés standard
        import unicodedata

        def normalize_key(key: str) -> str:
            """Normalise une clé en enlevant les accents et en minuscules"""
            # Enlever les accents
            key_normalized = unicodedata.normalize('NFD', key.lower())
            key_normalized = ''.join(c for c in key_normalized if unicodedata.category(c) != 'Mn')

            # Mapping spécifique pour les fautes de frappe courantes
            mapping = {
                "comprehension": "comprehension",
                "compréhension": "comprehension",
                "comprésentation": "comprehension",  # Faute de frappe
                "cooperation": "cooperation",
                "coopération": "cooperation",
                "progression": "progression",
                "total": "total"
            }

            return mapping.get(key_normalized, key_normalized)

        # Créer un nouveau dict avec les clés normalisées
        normalized_model = {}
        for key, value in details_model.items():
            normalized_key = normalize_key(key)
            # Si la clé normalisée existe déjà, garder la valeur la plus élevée (ou la première)
            if normalized_key not in normalized_model:
                normalized_model[normalized_key] = value
            else:
                # Si deux clés se normalisent vers la même, prendre la valeur numérique la plus élevée
                if isinstance(value, (int, float)) and isinstance(normalized_model[normalized_key], (int, float)):
                    normalized_model[normalized_key] = max(value, normalized_model[normalized_key])

        details_model = normalized_model

        # Valider que tous les champs sont présents
        required_fields = ["comprehension", "cooperation", "progression", "total"]
        for field in required_fields:
            if field not in details_model:
                details_model[field] = 5 if field != "total" else 15

        # Valider les valeurs (doivent être des nombres entre 0 et 10, ou total entre 0 et 30)
        for field in ["comprehension", "cooperation", "progression"]:
            if not isinstance(details_model[field], (int, float)) or not (0 <= details_model[field] <= 10):
                details_model[field] = 5

        # Recalculer total si nécessaire
        if "total" not in details_model or not isinstance(details_model["total"], (int, float)):
            details_model["total"] = details_model.get("comprehension", 5) + \
                                    details_model.get("cooperation", 5) + \
                                    details_model.get("progression", 5)
    else:
        # Valeurs par défaut si parsing échoue
        # Toujours afficher dans les logs Colab pour diagnostic
        print(f"⚠️ JSON non trouvé ou invalide dans réponse incrémentale")
        print(f"   Réponse brute (premiers 500 chars): {reponse_eval[:500]}")
        details_model = {"comprehension": 5, "cooperation": 5, "progression": 5, "total": 15}

    # Retourner selon le mode demandé
    if return_raw:
        return details_model, reponse_eval
    return details_model

# Endpoint FastAPI pour évaluation incrémentale
@app.post("/evaluate/incremental")
def evaluate_incremental(
    req: EvaluateRequest,
    debug: bool = Query(False, description="Activer le mode debug pour voir la réponse brute du modèle")
):
    """
    Évaluation légère au fil de l'eau (tous les 2 échanges)
    - Appelé par le frontend après chaque 2 échanges
    - Stocke les scores pour l'évaluation finale
    - Retourne seulement les scores (pas de message final)

    Paramètre debug (query string):
        ?debug=true - Retourne aussi la réponse brute du modèle dans la réponse HTTP
                      Exemple: POST /evaluate/incremental?debug=true
    """
    # Évaluer le dialogue récent (avec debug si demandé via query param ?debug=true)
    if debug:
        details_model, raw_response = evaluer_incremental(req.dialogue, debug=True, return_raw=True)
    else:
        details_model = evaluer_incremental(req.dialogue, debug=False, return_raw=False)
        raw_response = None

    # Stocker pour l'évaluation finale
    # Utiliser un ID simple basé sur le hash du dialogue
    # En production, utiliser un vrai ID de session
    dialogue_id = hash(req.dialogue)

    if dialogue_id not in incremental_scores:
        incremental_scores[dialogue_id] = []

    incremental_scores[dialogue_id].append({
        "scores": details_model,
        "exchange_count": len(incremental_scores[dialogue_id]) + 1
    })

    # Préparer la réponse
    response = {
        "scores": details_model,
        "exchange_count": len(incremental_scores[dialogue_id]),
        "accumulated": len(incremental_scores[dialogue_id]) > 0
    }

    # Ajouter la réponse brute si debug demandé
    if debug and raw_response:
        response["debug"] = {
            "raw_model_response": raw_response[:500],  # Premiers 500 chars
            "parsing_success": details_model.get("total", 0) != 15  # True si parsing réussi (pas valeurs par défaut)
        }

    return response

print("✅ Endpoint /evaluate/incremental créé pour évaluation au fil de l'eau")



NameError: name 'EvaluateRequest' is not defined